In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from scipy.misc import imread
import glob

# Labels for dataset: 0 for cat and 1 for dog 

In [81]:

labels_cat = np.zeros(12500)
labels_dog = np.ones(12500)

labels = np.concatenate([labels_cat,labels_dog]).reshape((-1,1))
labels = np.asarray(labels).astype('float32')
labels.shape

(25000, 1)

# Seperating cat and dog images from dataset

In [8]:
cat_files = glob.glob("train/train/cat*")
dog_files = glob.glob("train/train/dog*")

In [10]:
def convert_path_to_data(image_path):
    image = imread(image_path)
    #converting all image shape to have same size so that it concatenates in the end
    image = np.resize(image,[100,100,3])
    return image

In [11]:
cat_vector = []
for i in range(len(cat_files)):
    cat_vector.append(convert_path_to_data(cat_files[i]))
    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [12]:
dog_vector = []
for i in range(len(dog_files)):
    dog_vector.append(convert_path_to_data(dog_files[i]))
    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [13]:
dog_vector[10].shape

(100, 100, 3)

In [14]:
X = np.concatenate([cat_vector, dog_vector])

In [15]:
X.shape

(25000, 100, 100, 3)

# Spliting data into train and test

In [74]:
data_train, data_test, labels_train, labels_test = train_test_split(X, labels, test_size=0.1, random_state=1)

In [18]:
data_test.shape

(5000, 100, 100, 3)

# Training and network parameters

In [68]:
learning_rate = 0.0001
num_steps = 1000
batch_size = 32
num_classes = 1 
dropout = 0.25 

# Neural network architecture

In [69]:
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']
        
        conv1 = tf.layers.conv2d(x, 32, 3, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        conv3 = tf.layers.conv2d(conv2, 64, 3, activation=tf.nn.relu)
        conv3 = tf.layers.max_pooling2d(conv3, 2, 2)

        fc1 = tf.contrib.layers.flatten(conv3)
        fc1 = tf.layers.dense(fc1, 1024)

        fc2 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        return out 

# Model function

I used adagradoptimizer for optimization. I tried with adamoptimizer but loss overshoots and produces Nan value after some point. 

In [70]:
def model_fn(features, labels, mode):
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)

    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.sigmoid(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits_train, labels = tf.cast(labels, dtype=tf.float32)))
    optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    #train_op = tf.keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

# Estimator

In [78]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_tf_random_seed': None, '_model_dir': '/tmp/tmpt5w4girh', '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_task_type': 'worker', '_service': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_master': '', '_task_id': 0, '_evaluation_master': '', '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19500f4b00>}


# Converting data into float32 datatype because tensorflow does not allow int  

In [75]:
#data_train = np.array(data_train, dtype=float32)
data_train = data_train.astype(np.float32)    

In [76]:
data_test = data_test.astype(np.float32) 

# Defining the function for training

In [79]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': data_train}, y = labels_train,
    batch_size = batch_size, num_epochs = None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpt5w4girh/model.ckpt.
INFO:tensorflow:step = 0, loss = 7.1603565
INFO:tensorflow:global_step/sec: 34.4662
INFO:tensorflow:step = 100, loss = 4.097828 (2.903 sec)
INFO:tensorflow:global_step/sec: 36.33
INFO:tensorflow:step = 200, loss = 2.3863828 (2.753 sec)
INFO:tensorflow:global_step/sec: 36.4685
INFO:tensorflow:step = 300, loss = 1.2911124 (2.742 sec)
INFO:tensorflow:global_step/sec: 36.3772
INFO:tensorflow:step = 400, loss = 3.6303272 (2.749 sec)
INFO:tensorflow:global_step/sec: 36.3414
INFO:tensorflow:step = 500, loss = 1.2282616 (2.752 sec)
INFO:tensorflow:global_step/sec: 36.339
INFO:tensorflow:step = 600, loss = 1.5382468 (2.752 sec)
INFO:tensorflow:global_step/sec: 36.4486
INFO:tensorflow:step

# Evaluating the model performance on test set

In [80]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': data_test}, y=labels_test,
    batch_size=batch_size, shuffle=False)
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-19-15:53:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpt5w4girh/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-19-15:53:41
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.506, global_step = 1000, loss = 1.4775387


{'accuracy': 0.506, 'global_step': 1000, 'loss': 1.4775387}